In [6]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
#import kaleido
#from kaleido import write_fig_sync
ROOT = Path.cwd().parents[0]
DATA = ROOT / "data" / "processed" / "analytics_table.parquet"
FIGS = ROOT / "reports"/ "figures"
FIGS.mkdir(parents=True, exist_ok=True)
def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)
    #kaleido.write_fig_sync(fig, path=FIGS /"figure.png")

In [7]:
df = pd.read_parquet(DATA)
print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))
missing = df.isna().sum().sort_values(ascending=False).head(10)


rows: 100 cols: 18
order_id              string[python]
user_id               string[python]
amount                       Float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                 string[python]
dow                           object
hour                         float64
country                       object
dtype: object


<h3 style ="color: white"> EDA Questions </h3>


<ol>
  <li>Is revenue different by country?</li>
  <li>How does total revenue change by month?</li>
  <li>What is the distribution of order amount (winsorized)?</li>
</ol>

<h3> ------------------------------------------------------------------------</h3>
<h3 style ="color: white"> Revenue per country </h3>

In [8]:

revenue_country = (
df.groupby("country", dropna=False)
.agg(
n=("order_id","size"),
revenue=("amount","sum"),
aov=("amount","mean"),
)
.reset_index()
.sort_values("revenue")
)


#print(revenue_country)
fig = px.bar(revenue_country, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
fig.show()
save_fig(fig, FIGS / "revenue_by_country.png")
print("Run sccessful! check for (revenue_by_country.png) file in reports/figures folder")




Run sccessful! check for (revenue_by_country.png) file in reports/figures folder


<h3 style ="color: white">Interpertation: </h3>
<ol>
<li>Most of the money is coming from Saudi Arabia (SA). The chart shows it brings in more than *almost* four times the revenue compared to the United Arab Emirates (AE). </li>
<li>Looking at the list, there are far more rows for Saudi Arabia (SA). than for the United Arab Emirates (AE), meaning customers in Saudi Arabia are placing orders more frequently. </li>

</ol>

<h3 style ="color: white">Caveats:</h3>
<ol>
<li> Since Saudi Arabia has many more users in our data, the chart makes them look more profitable than AE.</li>
</ol>



<h3> ------------------------------------------------------------------------</h3>
<h3 style ="color: white"> Revenue trend (by day of the week) </h3>

In [9]:
dow_order = [
    "Sunday", "Monday", "Tuesday",
    "Wednesday", "Thursday", "Friday", "Saturday"
]

df["dow"] = pd.Categorical(
    df["dow"],
    categories=dow_order,
    ordered=True
)

revenue_dates = (
    df.groupby("dow", dropna=False)
      .agg(
          n=("order_id", "size"),
          revenue=("amount", "sum")
      )
      .reset_index()
      .sort_values("dow")
)

fig = px.line(
    revenue_dates,
    x="dow",
    y="revenue",
    title="Revenue by Day of Week"
)

fig.update_xaxes(title_text="Day of Week")
fig.update_yaxes(title_text="Revenue")
fig.show()
save_fig(fig, FIGS / "revenue_DOW.png")
print("Run successful! Check reports/figures/revenue_DOW.png")


C:\Users\gjjj3\AppData\Local\Temp\ipykernel_22672\3769161105.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Run successful! Check reports/figures/revenue_DOW.png


<h3 style ="color: white">Interpertation: </h3>
<ol>
<li>Thursday has the highest revenue: Revenue is highest on Thursdays, above $600.
  </li>
<li>  Revenue drops mid-week: Revenue is high on Sunday and Monday, then decreases on Tuesday and Wednesday.</li>
<li>Saturday has the lowest revenue: Revenue is lowest on Saturdays, below $200.</li>

</ol>

<h3 style ="color: white">Caveats:</h3>
<ol>
<li> This chart shows total revenue, but it doesn’t show if Thursday’s high revenue comes from many small orders or a few large ones. We can’t tell if Thursday is more popular or just more expensive.</li>
</ol>



<h3> ------------------------------------------------------------------------</h3>
<h3 style ="color: white"> Order Amount Distribution </h3>

In [10]:
fig = px.histogram(df, x="amount_winsor", nbins=15, title="Order amount disdistribution")
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
fig.show()
save_fig(fig, FIGS / "amount_hist_winsor.png")
print("Run sccessful! check for (amount_hist_winsor.png) file in reports/figures folder")




Run sccessful! check for (amount_hist_winsor.png) file in reports/figures folder


<h3 style ="color: white">Interpertation: </h3>
<ol>
<li> The tallest bar shows that the vast majority of orders (40+) are for very small amounts. </li>
<li> As the price goes up, the number of orders drops quickly. Very few people are making large purchases over $100.  </li>

</ol>

<h3 style ="color: white">Caveats:</h3>
<ol>
<li> This chart shows how many items were bought, not the total cash earned. While the "$0–$20" bar is the tallest, it takes ten of those small sales to equal the money from just one $200 sale. Focusing only on the tallest bar might make you miss where the real profit is coming from.</li>
</ol>



<h3> ------------------------------------------------------------------------</h3>
<h3 style ="color: white"> Bootstrap comparison </h3>

In [30]:

def bootstrap_diff_means(
    a: pd.Series,
    b: pd.Series,
    *,
    n_boot: int = 2000,
    seed: int = 0
) -> dict:
    rng = np.random.default_rng(seed)

    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())

    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }


d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))
print(bootstrap_diff_means(a, b, n_boot=2000, seed=0))

n_SA: 76 n_AE: 24
{'diff_mean': -0.0899122807017544, 'ci_low': -0.2807017543859649, 'ci_high': 0.07461622807017514}


<h3 style ="color: white">Interpertation: </h3>
<ol>
<li>On average, the refund rate in Saudi Arabia is about 9% lower than in the UAE (diff_mean: -0.09).  </li>
<li>  Because the Confidence Interval (CI) ranges from a negative number (-0.28) to a positive number (0.07), it includes zero. This means we cannot be 95% sure there is a real difference in refund behavior between the two countries; the result could just be due to chance. </li>

</ol>

<h3 style ="color: white">Caveats:</h3>
<ol>
<li>  The Saudi Arabia group is much more larger than the UAE group. Which can make the "difference" look more dramatic than it actually is.</li>
</ol>

